# Ch07. 머신 러닝(Machine Learning) 개요

# v05. 다중 입력에 대한 실습

- 독립 변수 $x$가 2개 이상인 선형 회귀와 로지스틱 회귀에 대해 학습
- 비용 함수와 옵티마이저 등을 사용하는 원리는 동일하다.

<br>

## 5.1 다중 선형 회귀

- 딥 러닝에서는 대부분의 입력들은 독립 변수가 2개 이상이다.
- 이는 모델을 직접 코딩하는 관점에서는 **입력 벡터의 차원이 2 이상**이라고 할 수 있다.

<br>

### 5.1.1 시험 점수 데이터

- $y$를 결정하는 데 있어 독립 변수가 3개인 선형 회귀를 풀어보자.
- 중간 고사, 기말 고사, 그리고 추가 점수를 어떤 공식을 통해 최종 점수를 계산한 데이터가 있다.

| Midterm($x1$) | Final($x2$) | Added point($x3$) | Score(\$1000)($y$) |
| :------------ | :---------- | :---------------- | :-------------- |
| 70            | 85          | 11                | 73              |
| 71            | 89          | 18                | 82              |
| 50            | 80          | 20                | 72              |
| 99            | 20          | 10                | 57              |
| 50            | 10          | 10                | 34              |
| 20            | 99          | 10                | 58              |
| 40            | 50          | 20                | 56              |


<br>

### 5.1.2 가설

- 이 경우 가설은 다음과 같다.

$
\qquad
H(X) = W_1 x_1 + W_2 x_2 + W_3 x_3 + b
$

<br>

### 5.1.3 데이터 분리 및 모델 학습

- 위의 데이터 중 상위 5개의 데이터만 훈련에 사용
- 나머지 2개는 테스트에 사용

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

X = np.array([[70,85,11],
              [71,89,18],
              [50,80,20],
              [99,20,10],
              [50,10,10]])
# 입력 벡터의 차원은 3이다. 즉, input_dim은 3이다.

y = np.array([73,82,72,57,34])
# 출력 벡터의 차원은 1이다. 즉, output_dim은 1이다.

model = Sequential()
model.add(Dense(1, input_dim=3, activation='linear')) # input_dim = 3 : 입력 벡터의 차원이 3

# 옵티마이저는 sgd(확률적 경사 하강법) 사용
# 학습률(learning rate)은 0.00001로 지정
sgd = optimizers.SGD(lr=0.00001)

# 손실 함수(loss function)는 평균제곱오차 mse 사용
model.compile(optimizer=sgd, loss='mse', metrics=['mse'])

model.fit(X, y, batch_size=1, epochs=2000, shuffle=False)

```
Epoch 1/2000
5/5 [==============================] - 0s 86ms/step - loss: 18534.6645 - mean_squared_error: 18534.6645
... 중략 ...
Epoch 2000/2000
5/5 [==============================] - 0s 2ms/step - loss: 0.0363 - mean_squared_error: 0.0363
```

<br>

### 5.1.4 예측

In [4]:
print(model.predict(X))

[[73.29252 ]
 [81.99197 ]
 [71.84287 ]
 [57.188465]
 [33.585   ]]


- 기존 데이터를 주고 예측을 시켰을 때 아직 정확하게 예측하는 건 아니지만 어느 정도 실제 값에 근접한 예측을 하는 것을 볼 수 있다.

<br>

### 5.1.5 평가

- 훈련할 때 사용하지 않았던 데이터를 가지고 예측 작업을 수행

In [5]:
X_test = np.array([[20, 99, 10],  # 58점 예측해야 함
                   [40, 50, 20]]) # 56점 예측해야 함
print(model.predict(X_test))

[[58.19531 ]
 [55.544582]]


<br>

## 5.2 다중 로지스틱 회귀

- $y$를 결정하는 데 있어 독립 변수 $x$가 2개인 로지스틱 회귀 문제

<br>

### 5.2.1 데이터

- 꽃받침(Sepal)의 길이와 꽃잎(Petal)의 길이와 해당 꽃이 A인지 B인지가 적혀져 있는 데이터가 있다.
- 새로 조사한 꽃받침의 길이와 꽃잎의 길이로부터 무슨 꽃인지 예측할 수 있는 모델을 만들고자 한다.
- 이 때 독립 변수 $x$는 2개가 된다.

| SepalLengthCm($x_1$) | PetalLengthCm($x_2$) | Species($y$) |
| :------------------- | :------------------- | :----------- |
| 5.1                  | 3.5                  | A            |
| 4.7                  | 3.2                  | A            |
| 5.2                  | 1.8                  | B            |
| 7                    | 4.1                  | A            |
| 5.1                  | 2.1                  | B            |

<br>

### 5.2.2 가설

- 이 경우 가설은 다음과 같다.

$
\qquad
H(X) = sigmoid \left( W_1 x_1 + W_2 x_2 + b \right)
$

<br>

### 5.2.3 OR 게이트 구현

- 독립 변수가 2개인 좀 더 간단한 예를 들고, 이를 케라스로 구현

- OR 게이트
  - 0 또는 1의 값을 입력으로 받는다.
  - 두 개의 입력 $x_1$, $x_2$ 중 하나라도 1이면 출력값 $y$가 1이 됨
  - 두 개의 입력이 0인 경우에만 출력값이 0이 됨

- 로지스틱 회귀를 통해 OR 게이트 구현

In [ ]:
import numpy as np

X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])
# 입력 벡터의 차원은 2이다. 즉, input_dim은 2이다.

y = np.array([0, 1, 1, 1])
# 출력 벡터의 차원은 1이다. 즉, output_dim은 1이다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

model = Sequential()
model.add(Dense(1, input_dim=2, activation='sigmoid')) # input_dim = 2 : 입력 차원은 2이다.

# 옵티마이저 : SGD(확률적 경사 하강법) 사용
# 비용 함수 : 이진 크로스 엔트로피(binary_crossentropy) 사용
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['binary_accuracy'])

model.fit(X, y, batch_size=1, epochs=800, shuffle=False)

```
Train on 4 samples
Epoch 1/800
4/4 [==============================] - 0s 69ms/sample - loss: 0.4338 - binary_accuracy: 1.0000
Epoch 2/800
4/4 [==============================] - 0s 3ms/sample - loss: 0.4322 - binary_accuracy: 0.7500
Epoch 3/800
4/4 [==============================] - 0s 3ms/sample - loss: 0.4305 - binary_accuracy: 0.7500

... 생략

Epoch 798/800
4/4 [==============================] - 0s 3ms/sample - loss: 0.1856 - binary_accuracy: 1.0000
Epoch 799/800
4/4 [==============================] - 0s 3ms/sample - loss: 0.1855 - binary_accuracy: 1.0000
Epoch 800/800
4/4 [==============================] - 0s 3ms/sample - loss: 0.1853 - binary_accuracy: 1.0000
<tensorflow.python.keras.callbacks.History at 0x7f5fca8612e8>
```

- 정확도는 100%가 나오고 있으므로 800회 정도로 학습을 멈춤

- 시그모이드 함수의 각 입력값에 대해서 출력값이 0.5보다 크고 작은 지를 확인

In [9]:
print(model.predict(X))

[[0.3628071 ]
 [0.89191985]
 [0.84965724]
 [0.98793864]]


- 입력이 둘 다 0, 0 인 경우를 제외하고 나머지 3개의 입력 쌍(pair)에 대해서는 전부 값이 0.5를 넘는 것을 볼 수 있다.

<br>

## 5.3 인공 신경망 다이어그램

- 다중 로지스틱 회귀를 뒤에서 배우게 되는 인공 신경망의 형태로 표현하면 다음과 같다.  
![](https://wikidocs.net/images/page/35821/multiplelogistic_regression.PNG)

- 이는 로지스틱 회귀를 일종의 인공 신경망 구조로 해석해도 무방함을 보여준다.

$
\qquad
\begin{align*}
y &= sigmoid(W_1 x_1 + W_2 x_2 + W_3 x_3 + \cdots + W_n x_n + b) \\
&= \sigma (W_1 x_1 + W_2 x_2 + W_3 x_3 + \cdots + W_n x_n + b)
\end{align*}
$